# Examining Racial Discrimination in the US Job Market

Tamara Monge

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
   

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
%matplotlib inline 
InteractiveShell.ast_node_interactivity = "all" 


In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


### 1. What test is appropriate for this problem? Does CLT apply?

Our objective is to determine whether or not race and callback rate are related. The sample size is large, but we do not know the population standard deviation, so I will use a 2-sample $t$-test. With a binomial population (e.g., "b" or "w") and a large *n*, the binomial can be approximated as a normal distribution and therefore the CLT does apply to our problem.  

### 2. What are the null and alternate hypotheses?

The null hypothesis is $H_0$: White- and black-sounding names receive the same callback rate.

The alternative hypothesis is $H_A$: White- and black-sounding names receive different callback rates.

### 3. Compute margin of error, confidence interval, and p-value.

In [5]:
# Separate into two datasets
white = data[data['race']=='w']
black = data[data['race']=='b']

# Number of resumes
n_w = len(white)
n_b = len(black)

# Proportion of callbacks
prop_w = np.sum(white['call']) / n_w
prop_b = np.sum(black['call']) / n_b
print('Percentage of callbacks for whites: ', prop_w)
print('Percentage of callbacks for blacks: ', prop_b)

# Difference in proportion of callbacks
prop_diff = prop_w - prop_b
print('Difference in percentage of callbacks: ', prop_diff)

# T-score
t_stat, p = stats.ttest_ind(white['call'],black['call'],equal_var=False)
print('t-statistic: ', t_stat)
print('p-value: ', p)

# Standard error
s_error = np.sqrt(white['call'].var()/n_w + black['call'].var()/n_b)

# Margin of Error
m_error = 1.96 * s_error
print('Margin of error:', m_error)

# Confidence Interval
c_int = prop_diff + (np.array([-1, 1]) * m_error)
print('Confidence interval:', c_int)

# p-value
p_value = stats.norm.cdf(-t_stat) * 2
print('p-value:', p_value)

prop_w/prop_b


Percentage of callbacks for whites:  0.0965092402464
Percentage of callbacks for blacks:  0.064476386037
Difference in percentage of callbacks:  0.0320328542094
t-statistic:  4.11470529086
p-value:  3.94294151365e-05
Margin of error: 0.0152584173807
Confidence interval: [ 0.01677444  0.04729127]
p-value: 3.87674401894e-05


1.4968152866242037

White-sounding names receive a callback rate of 0.0965, while black-sounding names receive only 0.0645. Could the difference we observe simply be a result of chance or does race affect callback percentage? 

To answer this, we compute the $t$-statistic under the null hypothesis and find $t$ = 4.11, with *p* = 0.000039. This high $t$ score and very small *p* value lead us to reject the null hypothesis and conclude that white- and black-sounding names receive different callback rates. 

The difference in callback rates is 0.0320. The margin of error on this difference is 0.0153.
The 95% confidence interval on the difference in callback rates 0.0168 - 0.0473. 

### 4. Write a story describing the statistical significance in the context or the original problem.
Racial discrimination has long plagued society and we have yet to fully understand its long reach and complex ramifications. Recently, researchers performed an experiment looking at factors (including race) that affect one's ability to receive employment in the US job market. The results from the survey show that white-sounding names received a callback percentage of 9.65%, while black-sounding names received a callback percentage of only 6.45%. In other words, for every interview granted to a black name, 1.5 interviews were granted to white names. We wanted to know if this observed difference could be random. To answer this, we conducted a two sample $t$-test. Our analysis shows that the race of a candidate does have a statistically significant effect on his/her callback success. 

### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Our analysis says that race is a factor in callback success; it does *not* say that race is the *most important* factor. In order to determine if race is the most important factor, we would need to account for all other variables (education, previous number of jobs, experience level, gaps in employment history, etc.). 